## Семинар 9. Retrieval‑Augmented Generation (RAG)

В этом уроке мы сделаем из LLM ассистента, который сможет отвечать на русском на произвольные вопросы о фильмах. Для этого мы будем использовать RAG и датасет с отзывами от Кинопоиска.

In [2]:
import numpy as np
import uuid
from tqdm.auto import tqdm

### Загрузка датасета и модели

Скачаем [датасет](https://huggingface.co/datasets/blinoff/kinopoisk) из HugingFace Hub. Он содержит больше 35 тысяч отзывов пользователей на различные фильмы.

In [3]:
from datasets import load_dataset

def process_dataset(sample):
    sample['content'] = sample['content'].replace('\xa0', ' ')
    return sample

dataset = load_dataset("blinoff/kinopoisk")['train']
dataset = dataset.map(process_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

kinopoisk.jsonl:   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36591 [00:00<?, ? examples/s]

Map:   0%|          | 0/36591 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['part', 'movie_name', 'review_id', 'author', 'date', 'title', 'grade3', 'grade10', 'content'],
    num_rows: 36591
})

In [5]:
dataset[0]['movie_name']

'Блеф (1976)'

In [6]:
dataset[0]['content']

'\n"Блеф» — одна из моих самых любимых комедий.\n\nЭтот фильм я наверно смотрел раз сто, нет я конечно блефую, я видел его куда больше. Не могу не выразить своё восхищение главными действующими лицами этого фильма. Начну с Адриано Челентано для которого как я считаю это лучшая роль в кино. Великолепный актёр, неплохой певец, странно что на его родине в Италии его песни мало кто слушает. Ну я думаю что и итальянцы и французы привыкли к тому, что у нас до сих их актёры популярней чем даже на своей родине. Да, такой вот парадокс. Челентано конечно профессионал своего дела, комик с серьёзным выражением лица. Он смешон ещё и потому, что одновременно так серъёзен. Адриано браво!\n\nА теперь несколько слов об Энтони Куине. Да тот самый горбун из Нотр-дама. Собор Парижской Богоматери, оригинальная версия, кто не смотрел рекомендую. С ним как-то приключилась одна интересная история. На съёмках одного из своих фильмов он то ли сломал, то ли подвихнул ногу, а роль требовала от него чтобы в одной 

Мы будем проводить все испытания с моделью [`Qwen/Qwen2-1.5B-Instruct`](https://huggingface.co/Qwen/Qwen2-1.5B-Instruct). Это вопросно-ответная модель на основе GPT, которая обучалась на большом количестве языков, в том числе на русском.

In [7]:
from transformers import pipeline
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

generation_pipeline = pipeline(
    "text-generation",
    model="Qwen/Qwen2-1.5B-Instruct",
    device=device,
    torch_dtype=torch.float16
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## Генерация без RAG

Проверим, насколько хорошо модель отвечает на вопросы без использования RAG.

In [16]:
query = 'В каких пяти фильмах играл Роберт де Ниро?'

In [ ]:
messages = [
    {"role": "user", "content": query},
]
output = generation_pipeline(messages, max_new_tokens=256, do_sample=True, temperature=0.9, top_p=0.7)

answer = output[0]['generated_text'][1]['content']

print(answer)

Роберт Де Ниро сыграл в следующих пяти фильмах:

1. "Богемская революция" (1968)
2. "Терминатор" (1984)
3. "Леон: Охотник на дичь" (1994)
4. "Американский квадрип" (1995)
5. "Суони" (2019)


Видим, что знаний модели не хватает. Все названия, кроме Терминатора, отсылают к несуществующим фильмам, а в Терминаторе Роберт де Ниро не играл.

##  Retrieval‑Augmented Generation

Попробуем улучшить качество модели с помощью RAG. Для этого нам сперва надо составить векторную базу данных. В качестве эмбеддинговой модели будем использовать [`intfloat/multilingual-e5-large`](https://huggingface.co/intfloat/multilingual-e5-large). Это большая мультиязычная модель на основе Encoder'a Трансформера.

In [8]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large", model_kwargs={'torch_dtype': torch.float16})

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Мы построим нашу базу данных на основе инструмента [`Qdrant`](https://github.com/qdrant/qdrant-client). В нем реализованы различные методы для поиска текстов, так что не придется писать ничего руками. Достаточно передать векторы эмбеддингов.

In [12]:
from qdrant_client import QdrantClient, models

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="kinopoisk_e5",
    on_disk_payload=True,
    vectors_config=models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE,
        on_disk=True
    ),
)

True

Для разбиения текста на куски используем `RecursiveCharacterTextSplitter` из библиотеки [`langchain`](https://github.com/langchain-ai/langchain). Мы будем делить текст на куски примерно по 1000 символов рекурсивно.

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

Собираем векторную базу данных.

In [14]:
for i in tqdm(range(len(dataset))):
    text_chunks = text_splitter.split_text(dataset[i]['content'])

    vectors = embedding_model.encode(text_chunks, normalize_embeddings=True, device=device).tolist()

    client.upsert(
        collection_name='kinopoisk_e5',
        points=[
            models.PointStruct(
                id=str(uuid.uuid4()),
                vector=vectors[j],
                payload={
                    'text': text_chunks[j],
                    'movie_name': dataset[i]['movie_name'][:-7],
                    'year': int(dataset[i]['movie_name'][-5:-1]),
                }
            )
            for j in range(len(text_chunks))
        ]
    )

  0%|          | 0/36591 [00:00<?, ?it/s]

Проверим, насколько хорошо ищутся похожие по смыслу тексты.

In [17]:
query_vector = embedding_model.encode(query, normalize_embeddings=True, device=device).tolist()

In [18]:
hits = client.search(
    collection_name="kinopoisk_e5",
    query_vector=query_vector,
    limit=5
)

In [19]:
[hit.payload for hit in hits]

[{'text': 'Роберт де Ниро сыграл потрясающую роль, передал всю боль, все счастье, все отчаяние больного.\n\nЭтот фильм должен посмотреть каждый. Возможно, кто-то сможет «пробудиться» к настоящей жизни.\n\n9 из 10',
  'movie_name': 'Пробуждение',
  'year': 1990},
 {'text': 'Потрясающий фильм. Великолепная игра Роберта Де Ниро. Изумительная по красоте и, как нельзя подходящая к этой кинокартине, музыка Эннио Морриконе. Этот шедевр стоит того, чтобы смотреть и смотреть не один раз.',
  'movie_name': 'Однажды в Америке',
  'year': 1983},
 {'text': 'Хороший фильм. На реальных событиях. Игра де Ниро восхищает. Робин Уильямс как обычно смешной и трагический.',
  'movie_name': 'Пробуждение',
  'year': 1990},
 {'text': 'После долгих и продуктивных лет работы в качестве актера, Роберт де Ниро решил попробовать себя и в режиссерском поприще. Первый фильм этого гениального человека собрал в Америке больше 17 миллионов долларов. Интригующее повествование о судьбе обычного молодого парня, которому п

Ура! Все полученные тексты связаны с Робертом де Ниро. Теперь обернем этот процесс поиска в функцию.

In [20]:
def semantic_search(client, query, limit=10):
    query_vector = embedding_model.encode(
        query, normalize_embeddings=True, device=device
    ).tolist()

    hits = client.search(
        collection_name="kinopoisk_e5",
        query_vector=query_vector,
        limit=limit
    )
    relevant_chunks = [hit.payload for hit in hits]

    return relevant_chunks

### RAG на отзывах

Для реализации RAG будем передавать в контекст модели отзывы, относящиеся к запросу.

In [21]:
def llm_answer(query, context):
    prompt = f"""
    Ты русскоязычный эксперт в области кинематографа. У тебя есть доступ к набору отзывов о фильмах, используй их, чтобы полно и точно ответить на следующий вопрос. Убедись, что ответ подробный, конкретный и непосредственно касается вопроса. Не добавляй информацию, которая не подтверждается предоставленными отзывами.

Вопрос:
{query}

Отзывы:
{context}
"""
    messages = [
        {"role": "user", "content": prompt},
    ]
    output = generation_pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.9, top_p=0.7)

    return output[0]['generated_text'][1]['content']

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query)
    context = ' ; '.join([f"Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context)

In [ ]:
print(predict(query))

Роберт де Ниро играл в следующих пяти фильмах:

1. "Загадочный человек" (The Departed)
2. "Огонь" (Heat)
3. "Тайны семьи" (The Departed)
4. "Алиса в стране чудес" (Alice in Wonderland)
5. "Мстители" (Captain America: Civil War)


Несмотря не подсказки, получилось так себе. Дело в том, что отзывы очень редко содержат сами названия фильмов. Попробуем добавить их тоже контекст.

### Добавляем названия фильмов

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query)
    context = ' ; '.join([f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context)

In [ ]:
print(predict(query))

Роберт де Ниро играл в следующих пяти фильмах:

1. Пробуждение (2006) - отзыв: "Хороший фильм. На реальных событиях. Игра де Ниро восхищает. Робин Уильямс как обычно смешной и трагический."
2. Однажды в Америке (1999) - отзыв: "Уникальный фильм. Игра Роберта Де Ниро не может не вызвать восхищения. Он действительно великолепен."
3. Пробуждение (2006) - отзыв: "Фильм о пробуждении души того человека, который пробудил физически больных 'хроников'."
4. Бронкская история (2005) - отзыв: "Богатейший актерский опыт Роберта Де Ниро с такими авторитетными режиссерами криминальных драм как Мартин Скорцезе, Брайан Де Пальма, Френсис Форд Коппола просто не мог пройти даром. И талантливый актер сделал не менее изящный, подобно десятку своих ролей, режиссерский дебют картиной 'Бронкская история'."
5. Схватка (2001) - отзыв: "Отдельно стоит упомянуть два эпизода фильма — перестрелку на улице и финальную разборку Аль Пачино с Де Ниро. Это действительно круто. Как в самом крутом боевике."


Теперь модель возвращает все фильмы, в которых Роберт де Ниро действительно играл.

## Reranker

Давайте попробуем улучшить метод, добавив переранжирование отзызов.

<img src="https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F906c3c0f8fe637840f134dbf966839ef89ac7242-3443x1641.png&w=3840&q=75" alt="drawing" width="1000"/>


Reranker – это языковая модель, принимающая два текста и возвращающая близость между ними.
В качестве реранкера мы будем использовать специальную мультиязычную модель, обученную для этих целей [`amberoad/bert-multilingual-passage-reranking-msmarco`](https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco).

In [ ]:
# ! pip install langchain_community

In [24]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [25]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='amberoad/bert-multilingual-passage-reranking-msmarco',
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)
sum([p.numel() for p in cross_encoder.client.model.parameters()])

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

167357954

In [26]:
def predict(query):
    selected_chunks = semantic_search(client, query, limit=50)

    texts = [f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks]
    scores = cross_encoder.score([(query, text) for text in texts])

    idxs = np.argsort(list(scores))[-10:]

    context = ' ; '.join([texts[i] for i in idxs])
    return llm_answer(query, context), context

In [27]:
answer, context = predict(query)

print(answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Роберт де Ниро играл в следующих пяти фильмах:

1. Бронкская история (1980)
2. Однажды в Америке (1987)
3. Военный ныряльщик (1995)
4. Пробуждение (2001)
5. Крестный отец 2 (2015)

Эти фильмы стали частью его многолетнего актерского репертуара и принесли ему множество наград, включая несколько Оскаров.


Стало действительно немного лучше.

In [28]:
context.split(' ; ')

['Название: Бронкская история. Отзыв: Богатейший актерский опыт Роберта Де Ниро с такими авторитетными режиссерами криминальных драм как Мартин Скорцезе, Брайан Де Пальма, Френсис Форд Коппола просто не мог пройти даром. И талантливый актер сделал не менее изящный, подобно десятку своих ролей, режиссерский дебют картиной «Бронкская история».\n\nСыграв также одну из второстепенных ролей, Роберт Де Ниро подарил необычный ему образ правильного и положительного героя, который из всех сил пытается вырастить в своем сыне мужество и справедливость. Ровно, как и герой Чазза Пальминтери, «крестного отца» итальянского квартала, которого, собственно больше боятся, чем любят. \n\nНо и герой Чазза Пальминтери по-своему прав, взяв на воспитание сына Лоренцо, он не утаивает сложность своего места под солнцем, и учит молодого парня в двух направлениях — школы и улицы одновременно.',
 'Название: Однажды в Америке. Отзыв: Естественно вы из всего вышеперечисленного не поймете, почему этот фильм настолько

Попробуем какие-нибудь другие вопросы. На вопрос о наиболее оскароносном фильме получили ответ Титаник. Модель ответила так из-за того, что из всех фильмов в _контексте_ у него больше всего оскаров.

In [ ]:
query = 'Какой фильм выиграл больше всего оскаров?'
answer, context = predict(query)

print(answer)

Фильм "Титаник" выиграл наибольшее количество Оскаров - 11.


In [ ]:
context.split(' ; ')

['Название: Амадей. Отзыв: Шикарные декорации, потрясающая музыка, отличная актерская игра, прекрасный сценарий оставляют просто великолепные ощущения после просмотра данного киношедевра. Да и 8 «Оскаров» и 3 номинации говорят сами за себя. \n\nОдин из наиболее успешных фильмов в истории Киноакадемии. Браво Милош Форман, браво актеры, браво Моцарт!',
 'Название: Король говорит!. Отзыв: После абсолютно немотивированных главных победителей «Оскара» последних двух лет — «Миллионера из трущоб» и «Повелителя бури» — объявления победителя 2011 года я ждал с изрядной долей опасения. К счастью, опасения оказались напрасными: посмотрев позднее фильм «Король говорит!» я, по крайней мере, могу понять логику рассуждений американских киноакадемиков и согласиться что лента об английском монархе — фильм достойный и стал лучшим по праву. Хотя, признаюсь, я больше бы обрадовался триумфу умного, психологически тонкого «Начала» Кристофера Нолана или простой, но вместе с тем гениальной в своей простоте «Ж

Благодаря промпту модель не отвечает на вопрос, если в контексте нет нужной информации!

In [ ]:
query = 'Сколько лет Тому Холланду?'
answer, context = predict(query)

In [ ]:
answer

'Извините, но я не могу предоставить вам информацию об возрасте Тома Холланда, так как все отзывы, которые вы указали, относятся к другим фильмам и не содержат информации о его возрасте.'

In [ ]:
context.split(' ; ')

['Название: Гран Торино, отзыв: Спросите у себя — сколько вы знаете режиссеров, способных снимать по два фильма в год? Сколько вы знаете режиссеров, способных снимать по два фильма в год, которые с легкостью претендуют в «топ-20 года». Сколько должно быть лет такому режиссеру? Клинту Иствуду уже семьдесят восемь, но его работоспособности позавидует любой начинающий режиссер, готовый трудиться годами без сна.',
 'Название: Начало, отзыв: И на закуску из мужчин я оставила Тома Харди. Тут ооо и только ооо. Потом я очнулась и стала следить за тем, как он играет. Вообще, появившись в костюме какого-то местного клоуняки или назовём этот образ — первый парень на селе — я недоверчиво покрутила головой. Потом гляжу — мой мальчик таки выбился в большое кино к 32 годам. Успех. Отменно отыграл и причём реально большую роль, показав, что способен быть и самостоятельным экшн-героем. Я у экрана просто сходила с ума. Хорошо, что хоть выла тихонько, а не в полную силу.\n\nБарышни тоже не подкачали. Пей

## Multi-Query

Попробуем расширить контекст, добавив перефразированные вопросы.

<img src="https://previews.dropbox.com/p/thumb/ACfUG85oSeshduKBOysw9GnEqnPtocIFtfpumlNs_wA7HVi0NqTSTm7iTBDGYgRUOnSuaDzVDdych734n2aHCD_YyGqdo3aQuep0mTc-JgvJIAR75cKD2cXeW9S5n5mtnGdHcr07bLlSSLjn5i0oytbn68WVV9S3oCNd_kiG8Vdfu1lqVxQQ1HFsJ6z_4jbURZgWLpzpbYpaS-ply7WNpdhpKVXxX_cj9GY_-DfyJwmjOU2dRXjP6IMCIzTiPXSo2zveJfnp0FGE2Qq_yaJ1NGuFLBuMqB6nvjdgQjfImSjUZRsDHxq49R-rFqB9h481SYp4kWzoItIO6cv7BFHNaCYx/p.png?is_prewarmed=true" alt="drawing" width="1000"/>

Для перефразирования будем использовать ту же модель, которой генерируем текст, но с новым промптом.

In [33]:
import re

def rephrase_query(query, n=3):
    prompt = f"""
Твоя задача написать {n} разных вариаций вопроса пользователя для того,
чтобы по ним получить релевантные документы из векторной базы данных.
Ты должен переформулировать вопрос с разных точек зрения.
Это поможет избавить пользователя от недостатков поиска похожих документов на основе расстояния.
Вопрос пользователя сфокусирован на теме кино.
Напиши ТОЛЬКО вариации вопроса и больше ничего, разделяя их символом новой строки \\n.
НЕ пиши ответ на сам вопрос.
-----------------
{query}

"""
    messages = [
        {"role": "user", "content": prompt},
    ]
    output = generation_pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.9, top_p=0.7)
    queries = output[0]['generated_text'][1]['content']

    return re.split(r'\n+', queries)


def predict(query):
    queries = rephrase_query(query, n=3)

    # retrieve documents for each query
    all_chunks = []
    for rephrased_query in queries:
        selected_chunks = semantic_search(client, rephrased_query, limit=5)
        all_chunks.extend(selected_chunks)

    context = [f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in all_chunks]

    # rerank documents
    scores = cross_encoder.score([(query, text) for text in np.unique(context)])
    idxs = np.argsort(list(scores))[-10:]
    context = ' ; '.join([context[i] for i in idxs])

    # generate answer
    answer = llm_answer(query, context)

    return answer, context, queries

In [34]:
query = 'Посоветуй легкую комедию'
answer, context, queries = predict(query)

In [35]:
print(answer)

С учетом всех представленных отзывов, можно предложить несколько легких комедий для просмотра:

1. "Как украсть миллион" - это отличная комедия, которая подойдет под любое настроение!
2. "День сурка" - фильм, который может вызвать много положительных эмоций и приносит радость.
3. "В погоне за счастьем" - это классический комедийный фильм, который всегда остается интересным.
4. "В джазе только девушки" - лучшая американская комедия, которую вы можете посмотреть.

Не забудьте также о просмотре таких комедий, как "Большая ржака!" или "Нереальный блокбастер", они тоже могут принести вам удовольствие.


Качество Multi-Query во много зависит от способности LLM перефразировать текст. Если она справляется плохо, то появятся нерелевантные запросы. Это можно исправить, добавив фильтрацию по соответствию входному запросу.

In [38]:
queries

['- Какие лучшие комедийные фильмы вы бы рекомендовали для просмотра?',
 '- Найдите мне фильм-комедию, который я бы мог смотреть дома в свободное время.',
 '- Пожалуйста, найдите мне хорошую комедию, которая подходит для моего любимого жанра.',
 '---',
 'Поиск похожих документов на основе расстояния может привести к тому, что пользователь получает слишком много результатов, которые не соответствуют его запросу. Вариант вопроса с точки зрения пользователя, который ищет легкую комедию, помогает избежать этого и обеспечивает более точный поиск.']

In [39]:
context.split(' ; ')

['Название: Наша Маша и Волшебный орех. Отзыв: faaaantik@gmail.com',
 'Название: В диких условиях. Отзыв: Потрясающая постановка того вопроса, которая нам показывает, что в жизни не стоит бежать от проблем, эти проблемы надо решать, пускай путем и сложным, но доступным, а не сложным и абсолютно бесполезным. И те слова, которые порою не могут отразить смысл, не всегда будут понятны другим людям, как и те действия, которые эти «бежащие» люди пытаются нам показать.\n\nЭтот человек не нашел счастья, да и вряд ли смог бы найти, потому что это счастье беспечности, ему просто было не с кем делить.\n\n9 из 10',
 'Название: Как украсть миллион. Отзыв: Очень хорошая комедия, которая подойдет под любое настроение! Советую всем!\n\n9 из 10',
 'Название: Как украсть миллион. Отзыв: Очень хорошая комедия, которая подойдет под любое настроение! Советую всем!\n\n9 из 10',
 'Название: В погоне за счастьем. Отзыв: -- Вердикт --',
 'Название: День сурка. Отзыв: Если у меня спрашивают какой твой любимый ф

## Фильтры

Для некоторых видов запросов можно добавить фильтры, чтобы в контекст не могли попадать документы, которые однозначно не подходят. Например, мы хотим узнать что-то про фильмы с привязкой к году. Если такой информации нет в самом отзыве, то при семантическом поиске мы не сможем ее использовать.

In [ ]:
def predict(query):
    selected_chunks = semantic_search(client, query, limit=10)
    context = ' ; '.join([
        f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}"
        for chunk in selected_chunks])

    return llm_answer(query, context), context

In [ ]:
query = 'Составь список пяти лучших мелодрам 1980-х годов'
answer, context = predict(query)

In [ ]:
print(answer)

Список пяти лучших мелодрам 1980-х годов:

1. "Привидение" (The Ghosts of Christmas Past, Present and Future) - Этот фильм, написанный Ричардом Лоренсом, стал классикой 80-х. Он имеет простую сюжетку о двух младших братах, которые пытаются открыть секреты своего отца, которого они знают лишь по его фотографиям.

2. "Крамер против Крамера" (Cramer vs Kramer) - Это комедийный фильм, основанный на реальных событиях. Фильм рассказывает историю молодого мужчины, который вынужден разделить обязанности с матерью своей дочери, которая не хочет видеть его после развода.

3. "Клуб 'Завтрак'" (Club Med) - Этот фильм является комедией с элементами драмы. Он рассказывает о группе друзей, которые вместе проводят выходные в клубе "Завтрак".

4. "Дневник памяти" (Memoirs of a Geisha) - Этот фильм представляет собой историю о девочки, которая живет в Японии и становится проституткой. Фильм также показывает ее отношения с другим девушкой.

5. "Звездный десант 3 Мародер" (Starship Troopers 3: Marauder) -

В этом примере модель сгенерировала не только не мелодрамы, но и фильмы неверных лет. Заметьте, что в годах присутствуют галлюцинации.

In [ ]:
def filtered_semantic_search(client, query, filter_years, limit=10):
    query_vector = embedding_model.encode(
        query, normalize_embeddings=True, device=device
    ).tolist()

    begin, end = filter_years
    hits = client.search(
        collection_name="kinopoisk_e5",
        query_vector=query_vector,
        limit=limit,
        query_filter=models.Filter(
            must=[models.FieldCondition(key="year", range=models.Range(gte=begin, lte=end))]
        ),
    )
    relevant_chunks = [hit.payload for hit in hits]

    return relevant_chunks

def predict(query, filter_years=None):
    if filter_years is not None:
        selected_chunks = filtered_semantic_search(client, query, filter_years=filter_years, limit=10)
    else:
        selected_chunks = semantic_search(client, query, limit=10)

    context = ' ; '.join([f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context), context

In [ ]:
answer, context = predict(query, filter_years=(1980, 1989))

In [ ]:
print(answer)

Список пяти лучших мелодрам 1980-х годов:

1. "Клуб 'Завтрак'"
2. "Ганди"
3. "Назад в будущее 2" 
4. "Назад в будущее"
5. "Однажды в Америке"

Эти фильмы отличаются своими уникальными сюжетами, великолепными актерскими играми, драматическими моментами и замечательным саундтреком. Каждый из них имеет свою уникальную историю, которая может вызвать волну эмоций у зрителя.
